In [1]:
import tensorflow as tf
from tensorflow import keras
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras import layers
from keras.layers import Activation
import time
from IPython import display

Using TensorFlow backend.


In [2]:
PATH = './animefacedataset'
file_list =[]
img_list = []
#load all data
def get_data(path=PATH):
    sub_dir = os.listdir(path)
    for i in sub_dir:
        curr = path+'/'+i
        if os.path.isdir(curr):
            get_data(curr)
        else:
            file_list.append(curr)
get_data()
#check for invalid data
j = 0
k = 0
start_load = time.time()
for i in file_list:
    j+=1
    try:
        tmp_img = cv2.imread(i)
        tmp_img = cv2.resize(tmp_img,(32,32))
        img_list.append(tmp_img)
        if(j%1000 == 0):
            print("load {} files, {:4.2g} sec for loading process".format(j,time.time()-start_load))
    except:
        if k == 0:
            print("list of uncompatible file(s)........")
        print(i)
        k+=1
    display.clear_output(wait=True)
d_in = np.array(img_list)
d_in = d_in.astype('float32')
d_in = (d_in-127.5)/127.5
print(len(d_in))

63565


In [3]:
validation = np.ones(len(d_in))
split = train_test_split(d_in, validation, test_size=0.1, random_state=30)
(trainX, testX, trainImgX, testImagesX) = split

In [4]:
#define discriminator model
img_shape = trainX[0].shape
def make_discriminator():
    model = keras.Sequential()
    inp=(keras.layers.Input(shape = img_shape))
    model=(keras.layers.Conv2D(32, 3))(inp)
    model=(keras.layers.LeakyReLU())(model)
    model=(keras.layers.Dropout(0.3))(model)
    model=(keras.layers.Conv2D(16,3,strides=(2,2)))(model)
    model=(keras.layers.LeakyReLU())(model)
    model=(keras.layers.Dropout(0.3))(model)    
    model=(keras.layers.Flatten())(model)
    model=(keras.layers.Dense(256))(model)
    model=(keras.layers.LeakyReLU())(model)
    model=(keras.layers.BatchNormalization(momentum= 0.8))(model)
    model=(keras.layers.Dense(1,activation='sigmoid'))(model)
    model = keras.models.Model(inp,model)
    tf.keras.backend.set_floatx('float64')
    return model


In [5]:
def make_generator_model(noise_size):
        inp = keras.Input(shape=(noise_size,))
        model = keras.layers.Dense(128 * 16 * 16)(inp)
        model = Activation("relu")(model)
        model = keras.layers.BatchNormalization(momentum= 0.8)(model)
        model = keras.layers.Reshape(
            (int(32/2), int(32/2), 128))(model)

        model = keras.layers.Conv2D(256, 5, padding='same')(model)
        model = Activation("relu")(model)
        model = keras.layers.BatchNormalization(momentum= 0.8)(model)
        model = keras.layers.Conv2DTranspose(
            256, 4, strides=2, padding='same')(model)
        model = Activation("relu")(model)
        model = keras.layers.BatchNormalization(momentum= 0.8)(model)
        model = keras.layers.Conv2D(256, 5, padding='same')(model)
        model = Activation("relu")(model)
        model = keras.layers.BatchNormalization(momentum= 0.8)(model)
        model = keras.layers.Conv2D(256, 5, padding='same')(model)
        model = Activation("relu")(model)
        model = keras.layers.BatchNormalization(momentum= 0.8)(model)
        model = keras.layers.Conv2D(
            3, 7, activation='tanh', padding='same')(model)
        model = keras.models.Model(inp, model)
        tf.keras.backend.set_floatx('float64')
        return model

In [26]:
#make loss function
BATCH_SIZE = 128
#generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)
gan_optimizer = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)

In [28]:
#train discriminator first!!!!!
#generated_image = generator(noise, training=False)
gen_ = make_generator_model(100)
noise = np.random.normal(size=(BATCH_SIZE,100))
gen_img = gen_.predict(noise)
for i in gen_img:
    np.append(trainX,i)
    np.append(trainImgX,0)
discriminator = make_discriminator()
discriminator.compile(optimizer = discriminator_optimizer,loss ='binary_crossentropy')
discriminator.fit(trainX, trainImgX,validation_data = (testX,testImagesX),epochs = 10,batch_size = 20 )
discriminator.save_weights("dis.h5")
#discriminator.fit(generated_image,fake)

Train on 57208 samples, validate on 6357 samples
Epoch 1/10
57208/57208 [==============================] - 42s 731us/sample - loss: 0.0183 - val_loss: 8.3383e-10
Epoch 2/10
57208/57208 [==============================] - 41s 713us/sample - loss: 6.7068e-10 - val_loss: 3.3752e-10
Epoch 3/10
57208/57208 [==============================] - 41s 715us/sample - loss: 3.6043e-10 - val_loss: 2.1116e-10
Epoch 4/10
57208/57208 [==============================] - 40s 707us/sample - loss: 2.4624e-10 - val_loss: 1.5605e-10
Epoch 5/10
57208/57208 [==============================] - 41s 710us/sample - loss: 1.8705e-10 - val_loss: 1.4142e-10
Epoch 6/10
57208/57208 [==============================] - 41s 722us/sample - loss: 1.4995e-10 - val_loss: 1.2005e-10
Epoch 7/10
57208/57208 [==============================] - 41s 715us/sample - loss: 1.2536e-10 - val_loss: 8.7086e-11
Epoch 8/10
57208/57208 [==============================] - 42s 727us/sample - loss: 1.0760e-10 - val_loss: 7.3755e-11
Epoch 9/10
57208/57

In [29]:
gan_inp = keras.Input(shape = (100,))
gan_gen = make_generator_model(100)
gan_dis = make_discriminator()
gan_dis.load_weights("dis.h5")
gan_out = gan_dis(gan_gen(gan_inp))
gan = keras.models.Model(gan_inp,gan_out)

gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')
gan_dis.compile(optimizer = discriminator_optimizer,loss ='binary_crossentropy')
noise = np.random.normal(size=(BATCH_SIZE,100))
#noise = np.random.normal(0,1,[BATCH_SIZE,100])
fake = np.zeros(BATCH_SIZE)
gan.summary()

Model: "model_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        [(None, 100)]             0         
_________________________________________________________________
model_18 (Model)             (None, 32, 32, 3)         8492803   
_________________________________________________________________
model_19 (Model)             (None, 1)                 809873    
Total params: 9,302,676
Trainable params: 9,302,164
Non-trainable params: 512
_________________________________________________________________


In [30]:
gan_dis.trainable =False
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')
gan.summary()

Model: "model_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        [(None, 100)]             0         
_________________________________________________________________
model_18 (Model)             (None, 32, 32, 3)         8492803   
_________________________________________________________________
model_19 (Model)             (None, 1)                 809873    
Total params: 9,302,676
Trainable params: 8,492,803
Non-trainable params: 809,873
_________________________________________________________________


In [31]:
gan.fit(noise,fake,epochs=50,batch_size=10)

Train on 128 samples
Epoch 1/50
128/128 [==============================] - 47s 370ms/sample - loss: 23.7860
Epoch 2/50
128/128 [==============================] - 47s 363ms/sample - loss: 23.5582
Epoch 3/50
128/128 [==============================] - 47s 368ms/sample - loss: 23.9185
Epoch 4/50
 30/128 [======>.......................] - ETA: 35s - loss: 23.5000

KeyboardInterrupt: 

In [ ]:
#gan.save_weights("gan.h5")
noise = np.random.normal(size=(1,100))
#noise = np.random.normal(0,1,[1,100])
pred = gan_gen.predict(noise)
# pred = np.array(pred[0]).reshape(
#             (64, 64, 3))
pred = (pred *127.5 + 125.5).astype('int')
print(pred)
plt.imshow(pred[0])
plt.show()